In [57]:
# Might need to install the packages below using 
# "pip install urllib" 
#  and 
# "pip install arxiv"
#  and 
# "pip install unidecode"
#  and 
# "pip install Levenshtein"
#  and 
# "pip install json"


import urllib.request
import arxiv
import unidecode
import json
from urllib.error import HTTPError
from urllib.parse import quote_plus, urlencode
from urllib.request import urlopen, Request
from Levenshtein import ratio, matching_blocks, editops
import string


def DOI_2_bib(doi):    
    doi_req = urllib.request.Request('http://dx.doi.org/' + doi)
    doi_req.add_header('Accept', 'application/x-bibtex')     
    with urllib.request.urlopen(doi_req) as f:
        output = f.read().decode()
        print(output)
        return output
    
    
def arXib_bib_format(paper):
    author_list = ' and '.join([str(author) for author in paper.authors])
    citation_key = str(paper.authors[0]).split(' ')[-1] + '_' + str(paper.published.year)
    citation_key = unidecode.unidecode(citation_key)
    return f'''@article{{{citation_key},
         title = {{{paper.title}}},
         author = {{{author_list.strip()} }},
         year = {{{paper.published.year}}},
         eprint = {{{paper.get_short_id()}}}, 
         archivePrefix = {{arXiv}},      
         primaryClass ={{{paper.categories[0]}}}
}}
    '''

def arXiv_2_bib(ID):
    search = arxiv.Search(id_list=[ID])
    paper = next(search.results())
    doi = paper.doi
    try: 
        return DOI_2_bib(doi)
    except: 
        print(arXib_bib_format(paper))  
        return arXib_bib_format(paper)
    
    
# adapted from https://github.com/OpenAPC/openapc-de/blob/master/python/import_dois.py    
def title_to_doi(title):
    
    api_url = "https://api.crossref.org/works?"
    params = {"rows": "5", "query.bibliographic": title}
    url = api_url + urlencode(params, quote_via=quote_plus)
    request = Request(url)

    try:
        ret = urlopen(request)
        content = ret.read()
        data = json.loads(content)
        items = data["message"]["items"]
        most_similar = {
                "crossref_title": "",
                "similarity": 0,
                "doi": ""
                }
        for item in items:
            if "title" not in item:
                continue
            title = item["title"].pop()
            result = {
                "crossref_title": title,
                "similarity": ratio(title.lower(), params["query.bibliographic"].lower()),
                "doi": item["DOI"]
            }
            if most_similar["similarity"] < result["similarity"]:
                most_similar = result
        print("Best match found: " + most_similar["crossref_title"])
        print("Similarity = " + str(most_similar["similarity"]))
        print()
        return most_similar["doi"]
    except HTTPError as httpe:
        print('No DOI matches found for this title')
        return 0
    
    
def getBib(ID):    
    if (len(ID.split()) > 1):  # means ID is actually the title
        return DOI_2_bib(title_to_doi(ID)) # title > doi > bib
    else: 
        try: 
            return DOI_2_bib(ID)
        except:
            return arXiv_2_bib(ID)
    

In [46]:
getBib('1410.0664');

@article{Fawzi_2015,
	doi = {10.1007/s00220-015-2466-x},
	url = {https://doi.org/10.1007%2Fs00220-015-2466-x},
	year = 2015,
	month = {sep},
	publisher = {Springer Science and Business Media {LLC}},
	volume = {340},
	number = {2},
	pages = {575--611},
	author = {Omar Fawzi and Renato Renner},
	title = {Quantum Conditional Mutual Information and Approximate Markov Chains},
	journal = {Communications in Mathematical Physics}
}


In [59]:
getBib('Machine learning the thermodynamic arrow of time');

Best match found: Machine learning the thermodynamic arrow of time
Similarity = 1.0

@article{Seif_2020,
	doi = {10.1038/s41567-020-1018-2},
	url = {https://doi.org/10.1038%2Fs41567-020-1018-2},
	year = 2020,
	month = {sep},
	publisher = {Springer Science and Business Media {LLC}},
	volume = {17},
	number = {1},
	pages = {105--113},
	author = {Alireza Seif and Mohammad Hafezi and Christopher Jarzynski},
	title = {Machine learning the thermodynamic arrow of time},
	journal = {Nature Physics}
}


In [49]:
getBib('10.1038/s41567-020-1018-2');

@article{Seif_2020,
	doi = {10.1038/s41567-020-1018-2},
	url = {https://doi.org/10.1038%2Fs41567-020-1018-2},
	year = 2020,
	month = {sep},
	publisher = {Springer Science and Business Media {LLC}},
	volume = {17},
	number = {1},
	pages = {105--113},
	author = {Alireza Seif and Mohammad Hafezi and Christopher Jarzynski},
	title = {Machine learning the thermodynamic arrow of time},
	journal = {Nature Physics}
}


In [51]:
getBib('10.1103/PhysRevLett.125.080601');

@article{Rossi_2020,
	doi = {10.1103/physrevlett.125.080601},
	url = {https://doi.org/10.1103%2Fphysrevlett.125.080601},
	year = 2020,
	month = {aug},
	publisher = {American Physical Society ({APS})},
	volume = {125},
	number = {8},
	author = {Massimiliano Rossi and Luca Mancino and Gabriel T. Landi and Mauro Paternostro and Albert Schliesser and Alessio Belenchia},
	title = {Experimental Assessment of Entropy Production in a Continuously Measured Mechanical Resonator},
	journal = {Physical Review Letters}
}


In [52]:
getBib('1610.02244');

@article{Al_Assam_2017,
	doi = {10.1088/1742-5468/aa7df3},
	url = {https://doi.org/10.1088%2F1742-5468%2Faa7df3},
	year = 2017,
	month = {sep},
	publisher = {{IOP} Publishing},
	volume = {2017},
	number = {9},
	pages = {093102},
	author = {S Al-Assam and S R Clark and D Jaksch},
	title = {The tensor network theory library},
	journal = {Journal of Statistical Mechanics: Theory and Experiment}
}
